<a href="https://colab.research.google.com/github/realyou0312/Bigdata-Gisa/blob/main/JY_Kaggle_Santander.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/빅분기 실기/Kaggle_Santander_분류/train.csv")
test = pd.read_csv("/content/drive/MyDrive/빅분기 실기/Kaggle_Santander_분류/test.csv")
print('dataset shape:', train.shape)
train.head(3)
test.shape

dataset shape: (76020, 371)


(75818, 370)

In [ ]:
cust_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76020 entries, 0 to 76019
Columns: 370 entries, var3 to TARGET
dtypes: float64(111), int64(259)
memory usage: 214.6 MB


In [ ]:
# print(cust_df['TARGET'].value_counts())
# unsatisfied_cnt = cust_df[cust_df['TARGET'] == 1].TARGET.count()
# total_cnt = cust_df.TARGET.count()
# print('unsatisfied 비율 {0:.2f}'.format((unsatisfied_cnt / total_cnt)))

In [ ]:
train.describe()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,...,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
count,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.0,76020.0,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,...,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000,7.602000e+04,7.602000e+04,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,7.602000e+04,76020.000000,76020.0,76020.000000,76020.000000,7.602000e+04,7.602000e+04,7.602000e+04,7.602000e+04,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000
mean,75964.050723,-1523.199277,33.212865,86.208265,72.363067,119.529632,3.559130,6.472698,0.412946,0.567352,3.160715,68.803937,113.056934,68.205140,113.225058,137.242763,68.618087,113.792410,140.403479,5.477676,0.011458,0.003762,0.0,0.0,0.958024,0.663760,0.000105,0.000026,0.032833,0.028598,0.067522,0.045462,0.052249,0.042936,0.041476,0.010168,0.009997,0.000026,0.000026,0.050855,...,1077.256756,1048.856447,68.275452,9.505287,124.620962,110.026575,3.997023e+03,613.534443,5.703008e+03,4.401002e+03,3639.419939,556.184178,4852.261814,3857.848542,771.227449,162.170439,9.569502e+02,7.509563e+02,0.175324,0.0,0.513023,0.344174,9.117181e+01,3.646318e+01,1.310316e+02,1.092169e+02,0.213071,0.001910,0.253907,0.186630,7.935824,1.365146,12.215580,8.784074,31.505324,1.858575,76.026165,56.614351,1.172358e+05,0.039569
std,43781.947379,39033.462364,12.956486,1614.757313,339.315831,546.266294,93.155749,153.737066,30.604864,36.513513,95.268204,319.605516,512.154823,531.897917,950.086398,697.712596,535.473750,953.578624,712.767240,465.391149,0.106425,0.061221,0.0,0.0,0.200535,0.472425,0.010258,0.005129,0.178202,0.166674,0.250925,0.208316,0.222531,0.202714,0.199390,0.100325,0.099486,0.005129,0.005129,0.219703,...,9614.906985,8189.948852,1733.838226,519.389157,2205.249804,1935.305713,3.777314e+04,9292.752726,4.620254e+04,3.550718e+04,26359.174223,7182

In [ ]:
train['var3'].replace(-999999,2, inplace=True)
train.drop('ID',axis=1 , inplace=True)

## 피쳐와 레이블 분리

In [ ]:
X = train.iloc[:, :-1]
y = train.iloc[:, -1]
print('피처 데이터 shape:{0}'.format(X.shape))

피처 데이터 shape:(76020, 369)


In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)
X['PCA1'] = X_pca[:,0]
X['PCA2'] = X_pca[:,1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_cnt = y_train.count()
test_cnt = y_test.count()
print('학습 세트 Shape:{0}, 테스트 세트 Shape:{1}'.format(X_train.shape , X_test.shape))

print(' 학습 세트 레이블 값 분포 비율')
print(y_train.value_counts()/train_cnt)
print('\n 테스트 세트 레이블 값 분포 비율')
print(y_test.value_counts()/test_cnt)

학습 세트 Shape:(60816, 371), 테스트 세트 Shape:(15204, 371)
 학습 세트 레이블 값 분포 비율
0    0.96052
1    0.03948
Name: TARGET, dtype: float64

 테스트 세트 레이블 값 분포 비율
0    0.960076
1    0.039924
Name: TARGET, dtype: float64


## 모델링 : XGBoost

In [ ]:
xgb_clf = XGBClassifier(n_estimators=500, random_state=42)

xgb_clf.fit(X_train, y_train, early_stopping_rounds=100,
            eval_metric="auc", eval_set=[(X_train, y_train), (X_test, y_test)])

xgb_roc_score = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:,1],average='macro')
print('ROC AUC: {0:.4f}'.format(xgb_roc_score))

[0]	validation_0-auc:0.802515	validation_1-auc:0.793184
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 100 rounds.
[1]	validation_0-auc:0.813242	validation_1-auc:0.802802
[2]	validation_0-auc:0.812548	validation_1-auc:0.800996
[3]	validation_0-auc:0.816391	validation_1-auc:0.805699
[4]	validation_0-auc:0.821967	validation_1-auc:0.812246
[5]	validation_0-auc:0.821599	validation_1-auc:0.81351
[6]	validation_0-auc:0.821695	validation_1-auc:0.811729
[7]	validation_0-auc:0.818716	validation_1-auc:0.810634
[8]	validation_0-auc:0.821822	validation_1-auc:0.8103
[9]	validation_0-auc:0.82387	validation_1-auc:0.815418
[10]	validation_0-auc:0.823313	validation_1-auc:0.81321
[11]	validation_0-auc:0.823563	validation_1-auc:0.814485
[12]	validation_0-auc:0.823809	validation_1-auc:0.815316
[13]	validation_0-auc:0.824289	validation_1-auc:0.816086
[14]	validation_0-auc:0.824429	validation_1-auc:0.815735
[1

In [ ]:
xgb_clf = XGBClassifier(n_estimators=100)

params = {'max_depth':[5, 7] , 'min_child_weight':[1,3] ,'colsample_bytree':[0.5, 0.75] }

gridcv = GridSearchCV(xgb_clf, param_grid=params, cv=3)
gridcv.fit(X_train, y_train, early_stopping_rounds=30, eval_metric="auc",
           eval_set=[(X_train, y_train), (X_test, y_test)])

print('GridSearchCV 최적 파라미터:',gridcv.best_params_) 

xgb_roc_score = roc_auc_score(y_test, gridcv.predict_proba(X_test)[:,1], average='macro')
print('ROC AUC: {0:.4f}'.format(xgb_roc_score))

[0]	validation_0-auc:0.729396	validation_1-auc:0.692918
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 30 rounds.
[1]	validation_0-auc:0.808017	validation_1-auc:0.788334
[2]	validation_0-auc:0.80383	validation_1-auc:0.780812
[3]	validation_0-auc:0.813957	validation_1-auc:0.790962
[4]	validation_0-auc:0.822253	validation_1-auc:0.803007
[5]	validation_0-auc:0.817544	validation_1-auc:0.795433
[6]	validation_0-auc:0.818023	validation_1-auc:0.794015
[7]	validation_0-auc:0.823564	validation_1-auc:0.799583
[8]	validation_0-auc:0.827417	validation_1-auc:0.804149
[9]	validation_0-auc:0.828368	validation_1-auc:0.804829
[10]	validation_0-auc:0.83184	validation_1-auc:0.807626
[11]	validation_0-auc:0.836266	validation_1-auc:0.814045
[12]	validation_0-auc:0.837546	validation_1-auc:0.815118
[13]	validation_0-auc:0.835739	validation_1-auc:0.813076
[14]	validation_0-auc:0.834635	validation_1-auc:0.810902
